# Gemma Normalization

Erstellen der Output-Klasse

In [1]:
from pydantic import BaseModel
from typing import Optional


class NormalizationItem(BaseModel):
    original: Optional[str] = None
    Normalisiert: str

`reisen02_filtered.json`, die zuvor in `corpus_creation.ipynb` erstellt wurde, wird als `input_json` definiert.

In [2]:
import json

input_json = "reisen02_filtered.json"

## Normalisierung

Berechnungsdauer: 370 Minuten

In [ ]:
import ollama

# JSON-Datei laden
with open(input_json, "r", encoding="utf-8") as f:
    data = json.load(f)

sentences = data["sentences"]

results = []

# durch die Sätze iterieren
for sentence in sentences:
    # orig_text wird am Ende des Prompts an das Modell übergeben.
    orig_text = sentence["orig_sentence"]["text"]
    norm_text = sentence["norm_sentence"]["text"]
    sentence_id = sentence["sentence_id"]

    # Prompt für das Modell erstellen
    prompt = f"""
        Du bist ein KI-Assistent, der historische deutsche Texte normalisiert.
        Deine Aufgabe ist es, die Othografie eines übergebenen Textes nach folgenden Regeln zu modernisieren:
        - Ersetze veraltete Buchstaben und Zeichennutzung wie (ſ,uͤ,aͤ,oͤ,y,th), durch moderne Entsprechungen (s,ü,ä,ö,i,t).
        Beispiel:[
        "Satz": "Zwey ſchoͤne Anhoͤhen ſahen wir im suͤdlichen Theil.
        "response": "Zwei schöne Anhöhen sahen wir im südlichen Teil."]
        - Passe außschließlich die Orthografie an und ersetze keine Wörter.
        Beispiel:[
        "Satz": "Einige Tage hernach ſa- hen wir Beſansſegel, auf dem Waſſer ſegeln."
        "response": "Einige Tage hernach sahen wir Besanssegel, auf dem Wasser segeln."]
        - Verändere keine unbekannten Wörter, wenn die Orthografie an sich modern ist.
        Beispiel:[
        "Satz": "Wir ſegelten ab, nachdem Capitain Morland auf dem Commandeurſchiffe Boven Kerkerpolder das Zeichen gegeben hatte."
        "response": "Wir segelten ab, nachdem Capitain Morland auf dem Commandeurschiffe Boven Kerkerpolder das Zeichen gegeben hatte."]
        - Verändere keine Namen und Eigennamen.
        Beispiel:[
        "Satz": "Unterwegs ſah ich die Capſche Miſtel (Viſcus capenſis), ein paraſitiſches Gewaͤchs."
        "response": "Unterwegs sah ich die Capsche Mistel (Viscus capensis), ein parasitisches Gewächs"]

    "Satz": {orig_text}
    """

    # Modell aufrufen
    response = ollama.chat(
    model='gemma3:12b',
    messages=[
        {"role": "user",
         "content": prompt,
         "temperature": 0.2
        }
    ],
    format=NormalizationItem.model_json_schema(),
    )

    result = NormalizationItem.model_validate_json(response.message.content)
    print(response.message.content)


    results.append({
    "sentence_id": sentence_id,
    "model_output": result.Normalisiert,
    "norm_sentence": norm_text
    })

output_file = "normalization_results.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

In dem Normaliiserungsskript wurden die Originalsätze nicht zu der `.json`inzugefügt. Das wird hier nachgeholt.

In [5]:
# Ergebnisse aus results2.json laden
results_file = "results2.json"
with open(results_file, "r", encoding="utf-8") as f:
    results = json.load(f)

# Erweiterung der Ergebnisse um die originalen Sätze
results_expanded = []

for sentence, result in zip(sentences, results):
    expanded_entry = {
        "sentence_id": result["sentence_id"],
        "orig_sentence": sentence["orig_sentence"]["text"],
        "model_output": result["model_output"],
        "norm_sentence": result["norm_sentence"]
    }
    results_expanded.append(expanded_entry)

# Ergebnisse in einer neuen Datei speichern
expanded_output_file = "results_expanded.json"
with open(expanded_output_file, "w", encoding="utf-8") as f:
    json.dump(results_expanded, f, ensure_ascii=False, indent=4)

Um Ground Truth und die Normalisierung des Sprachmodells angemessen vergleichen zu können, werden nur die Sätze in eine jeweilige `.txt` übernommen, in denen auch Änderungen zwischen der Ground Truth und der Normaliiserung stattfinden. So wird vermieden, dass die Ergebnisse zu positiv ausfallen. Die bieden `.txt`s können anschließend dinglehopper für die Berechnung der `WER`s und `CER`s übergeben werden.

In [6]:
# Dateien für die Normalisierung erstellen
gt_normalized_file = "gt_normalized.txt"
llm_normalized_file = "llm_normalized.txt"

transferred_count = 0
ignored_count = 0

with open(gt_normalized_file, "w", encoding="utf-8") as gt_file, open(llm_normalized_file, "w", encoding="utf-8") as llm_file:
    for entry in results_expanded:
        orig_sentence = entry["orig_sentence"]
        norm_sentence = entry["norm_sentence"]
        model_output = entry["model_output"]

        # Nur Sätze übertragen, bei denen sich etwas zwischen Original und Normalisierung ändert
        if orig_sentence != norm_sentence:
            gt_file.write(norm_sentence + "\n")
            llm_file.write(model_output + "\n")
            transferred_count += 1
        else:
            ignored_count += 1

# Ausgabe der Anzahl der übertragenen und ignorierten Sätze
print(f"Anzahl der übertragenen Sätze: {transferred_count}")
print(f"Anzahl der ignorierten Sätze: {ignored_count}")

Anzahl der übertragenen Sätze: 5652
Anzahl der ignorierten Sätze: 406


Da sich aber auch in Sätzen, die keine Veränderungen erfahren sollen, Fehler befinden können, werden sie noch einmal überprüft.

In [7]:
# Zählen der Sätze, bei denen alle drei Variablen identisch sind
identical_count = 0

for entry in results_expanded:
    orig_sentence = entry["orig_sentence"]
    norm_sentence = entry["norm_sentence"]
    model_output = entry["model_output"]

    # Überprüfen, ob alle drei Variablen identisch sind
    if orig_sentence == norm_sentence == model_output:
        identical_count += 1

# Ausgabe der Anzahl der identischen Sätze
print(f"Anzahl der Sätze, bei denen orig_sentence, norm_sentence und model_output identisch sind: {identical_count}")

Anzahl der Sätze, bei denen orig_sentence, norm_sentence und model_output identisch sind: 332


Nachfolgend werden alle Sätze ausgegeben, bei denen Original und Normalisierung identisch sind, aber der Modelloutput etwas verändert hat.

In [9]:
# Sätze finden, bei denen orig_sentence und norm_sentence identisch sind, aber model_output abweicht
differing_model_output = []

for entry in results_expanded:
    orig_sentence = entry["orig_sentence"]
    norm_sentence = entry["norm_sentence"]
    model_output = entry["model_output"]

    # Überprüfen, ob orig_sentence und norm_sentence identisch sind, aber model_output abweicht
    if orig_sentence == norm_sentence and norm_sentence != model_output:
        differing_model_output.append(entry)

# Ausgabe der Sätze
print("Sätze, bei denen orig_sentence und norm_sentence identisch sind, aber model_output abweicht:")
for entry in differing_model_output:
    #print(f"sentence_id: {entry['sentence_id']}")
    print(f"orig_sentence: {entry['orig_sentence']}")
    print(f"norm_sentence: {entry['norm_sentence']}")
    print(f"model_output: {entry['model_output']}")
    print()

Sätze, bei denen orig_sentence und norm_sentence identisch sind, aber model_output abweicht:
orig_sentence: 14.
norm_sentence: 14.
model_output: vierzehn.

orig_sentence: 32.
norm_sentence: 32.
model_output: Ich verstehe. Bitte geben Sie mir den zu normalisierenden Text.

orig_sentence: 46.
norm_sentence: 46.
model_output: Die ſchon beſagte ſtarke Bewegung, welche wir von der See zuͤrük an das Ufer treibt, war nunmehr verſchwunden.

orig_sentence: 4. Rothmannia, et nytt Örte-Genus.
norm_sentence: 4. Rothmannia, et nytt Örte-Genus.
model_output: 4. Rothmannia, et neue Örte-Genus.

orig_sentence: 6. Anmärkningar vid Hydnora africana.
norm_sentence: 6. Anmärkningar vid Hydnora africana.
model_output: 6. Anmerkungen bei Hydnora africana.

orig_sentence: 7.
norm_sentence: 7.
model_output: Sie wurden von dem Hofrath und dem Advocaten, welche in dem Theil des Hauses gewohneten, in freundlicher Weiſe aufgenommen.

orig_sentence: 13. Noctua Serici, enny Silkes-malk.
norm_sentence: 13. Noctua Se